In [18]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [19]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

llm_model = "gpt-3.5-turbo"
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [42]:
url_contact = os.environ["URL_CONTACT"]
url_newsletter = os.environ["URL_NEWSLETTER"]
system_msg = "You are a world class interior designer that specializes in the Transitional Design style.  You have a knack for explaining concepts related to interior design in a way that is accessible and engaging."
human_msg = """Please write a blog post with 2 sections in the first person with a bit of humor about "{input}" 
with a call to action at the end to contact me for a free consultation or to sign up for my newsletter. 
Include an introduction that indicates that this article is part of my series of interior design blog posts called the "{series_title}" series.
The link to my newsletter is {url_newsletter}
The "Contact Me" link should have a URL of {url_contact}
I would like you to craft each section with enough detail for three paragraphs each.
I would like each section, including the introduction and the conclusion should be preceded with a "####" delimiter.  
Please list a good seo keyphrase and a meta description at the end. 
Avoid using the passive voice in any sentence.  Sentences should be no more than 20 words long."""

In [39]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessage,
    HumanMessagePromptTemplate,
    PromptTemplate,
)
system_message_prompt = SystemMessage(
    content=(system_msg)
)
human_message_prompt = HumanMessagePromptTemplate(
        prompt=PromptTemplate(
            template=human_msg,
            input_variables=["input", "series_title", "url_contact", "url_newsletter"],
        )
    )

In [40]:
template = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [22]:
prompt=template.format_messages(
    input="How to place picture frames on a bookshelf for maximum effect.",
    series_title="Do It Yourself"
)

In [24]:
llm(prompt)

AIMessage(content='#### Introduction\n\nWelcome to the "{series_title}" series, where we dive into the world of interior design with a touch of humor and a lot of expertise. In this installment, we\'ll explore the concept of "{input}" and how it can transform your space. If you\'re looking for a fresh, sophisticated style that blends the best of traditional and contemporary design, then you\'re in for a treat. So grab a cup of coffee and let\'s get started!\n\n#### Section 1: What is "{input}"?\n\nYou\'ve probably heard the term "{input}" thrown around, but what exactly does it mean? At its core, "{input}" is all about balance. It\'s the perfect marriage between traditional and contemporary design, creating a harmonious and timeless look. Essentially, "{input}" takes the best elements from both styles and combines them to create a fresh and inviting space.\n\nIn a "{input}" design, you\'ll find clean lines and simple forms, just like you would in a contemporary space. However, it also 

In [ ]:
chain = LLMChain(llm=llm, prompt=template)
print(chain.run({
    'input': "How to place picture frames on a bookshelf for maximum effect.",
    'series_title': "Do It Yourself"
    }))